In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("bano-21.csv", sep=",", header=None)
df.head()

,0,1,2,3,4,5,6,7
0,210010022-1,1,Rue de Citeaux,21700,Agencourt,BAN,47.128003,4.981303
1,210010022-10,10,Rue de Citeaux,21700,Agencourt,BAN,47.127759,4.982275
2,210010022-11,11,Rue de Citeaux,21700,Agencourt,BAN,47.127800,4.982668
3,210010022-12,12,Rue de Citeaux,21700,Agencourt,BAN,47.127730,4.982469
4,210010022-13,13,Rue de Citeaux,21700,Agencourt,BAN,47.127721,4.983235


In [3]:
df.drop(columns=[0, 5], inplace=True)
df

,1,2,3,4,6,7
0,1,Rue de Citeaux,21700,Agencourt,47.128003,4.981303
1,10,Rue de Citeaux,21700,Agencourt,47.127759,4.982275
2,11,Rue de Citeaux,21700,Agencourt,47.127800,4.982668
3,12,Rue de Citeaux,21700,Agencourt,47.127730,4.982469
4,13,Rue de Citeaux,21700,Agencourt,47.127721,4.983235
...,...,...,...,...,...,...
218665,7,Rue du Tertre,21290,Voulaines-les-Templiers,47.820919,4.778251
218666,9,Rue du Tertre,21290,Voulaines-les-Templiers,47.821701,4.777641
218667,1,Route des Marots,21290,Voulaines-les-Templiers,47.814176,4.766458
218668,2,Route des Marots,21290,Voulaines-les-Templiers,47.815451,4.773532


In [4]:
df.rename(columns={1: "numero_voie", 2: "libelle_voie", 3: "code_postal", 4: "nom_commune", 6: "lon", 7: "lat"}, inplace=True)
df

,numero_voie,libelle_voie,code_postal,nom_commune,lon,lat
0,1,Rue de Citeaux,21700,Agencourt,47.128003,4.981303
1,10,Rue de Citeaux,21700,Agencourt,47.127759,4.982275
2,11,Rue de Citeaux,21700,Agencourt,47.127800,4.982668
3,12,Rue de Citeaux,21700,Agencourt,47.127730,4.982469
4,13,Rue de Citeaux,21700,Agencourt,47.127721,4.983235
...,...,...,...,...,...,...
218665,7,Rue du Tertre,21290,Voulaines-les-Templiers,47.820919,4.778251
218666,9,Rue du Tertre,21290,Voulaines-les-Templiers,47.821701,4.777641
218667,1,Route des Marots,21290,Voulaines-les-Templiers,47.814176,4.766458
218668,2,Route des Marots,21290,Voulaines-les-Templiers,47.815451,4.773532


In [5]:
# Merge 'lon' and 'lat' columns into a new 'location' column
df['location'] = df['lon'].astype(str) + ',' + df['lat'].astype(str)
df.drop(["lon", "lat"], inplace=True, axis=1)
df.head()

,numero_voie,libelle_voie,code_postal,nom_commune,location
0,1,Rue de Citeaux,21700,Agencourt,"47.128003,4.981303"
1,10,Rue de Citeaux,21700,Agencourt,"47.127759,4.982275"
2,11,Rue de Citeaux,21700,Agencourt,"47.1278,4.982668"
3,12,Rue de Citeaux,21700,Agencourt,"47.12773,4.982469"
4,13,Rue de Citeaux,21700,Agencourt,"47.127721,4.983235"


In [6]:
df['index'] = range(len(df))

In [7]:
cols = df.columns.to_list()
cols.insert(0, cols.pop(cols.index("index")))
df = df[cols]

In [8]:
df.to_csv("bano-21_clean.csv", sep=",", index=False)

In [9]:
from elasticsearch import Elasticsearch

# Connect to the Elasticsearch cluster
es = Elasticsearch(
    ["http://localhost:9200"],  # Replace with your Elasticsearch cluster URL
    basic_auth=("elastic", "<ES_PASSWORD>"),  # Replace with your username and password if authentication is enabled
)

# Check if the connection is established
if es.ping():
    print("Connected to Elasticsearch")
else:
    print("Could not connect to Elasticsearch")

Connected to Elasticsearch


In [10]:
from elasticsearch import helpers

# Convert DataFrame to list of dictionaries
docs = df.to_dict(orient='records')

# Create a list of actions for the bulk helper
actions = [
    {
        "_index": "bano_21_through_python",
        "_id": doc['index'],
        "_source": doc
    }
    for doc in docs
]

# Use the bulk helper to send the actions to Elasticsearch
helpers.bulk(es, actions)


(218670, [])

In [11]:
resp = es.search(
    index="bano_21_through_python",
    query={
        "bool":{
            "must": [
                {
                    "match" : {
                        "code_postal": "21000"
                    },
                },
                {
                    "match" : {
                        "libelle_voie": "lafayette"
                    }
                } 
                ]
        }
        
    },
)
resp.raw

{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 59, 'relation': 'eq'},
  'max_score': 10.306333,
  'hits': [{'_index': 'bano_21_through_python',
    '_id': '77547',
    '_score': 10.306333,
    '_source': {'index': 77547,
     'numero_voie': '1',
     'libelle_voie': 'Rue Lafayette',
     'code_postal': 21000,
     'nom_commune': 'Dijon',
     'location': '47.334149,5.044244'}},
   {'_index': 'bano_21_through_python',
    '_id': '77548',
    '_score': 10.306333,
    '_source': {'index': 77548,
     'numero_voie': '10',
     'libelle_voie': 'Rue Lafayette',
     'code_postal': 21000,
     'nom_commune': 'Dijon',
     'location': '47.333946,5.045121'}},
   {'_index': 'bano_21_through_python',
    '_id': '77549',
    '_score': 10.306333,
    '_source': {'index': 77549,
     'numero_voie': '11',
     'libelle_voie': 'Rue Lafayette',
     'code_postal': 21000,
     'nom_commune': 'Dijon',
     'location': '4

![Screenshot Dashboard](img/image.png)